# Calculating the Absolute Magnitude of a Population Feature

This is a notebook that calculates the absolute magnitude of a population feature using the MIST isochrones. These pre-calculated Magnitudes will then be used in the `time_to_feature.ipynb` notebook in the main directroy to calculate the exposure time required to reach a given feature for a given observation (distance to the population, its metallicity, its age, etc.).

In [1]:
# all we need is numpy, matplotlib
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# These are just preferences for making plots
# If you are getting errors from matplotlib, feel free to comment this out
plt.rc('font', **{'size': 8, 'sans-serif': ['Helvetica'], 'family': 'sans-serif'})
plt.rc('legend',**{'fontsize':6})                                             
plt.rc("text.latex", preamble=[
    "\\usepackage[T1]{fontenc}\\usepackage{amsmath}"])
plt.rc("text", usetex=True)
plt.rc('ps', usedistiller='xpdf')
plt.rc('savefig', **{'dpi': 300})

Like the other notebooks, we begin by defining a lot of the same variables.

In [3]:
# First setting the range of metallicities and ages that are covered, these are our default values
metlist = np.array([0.5,0.25,0.0,-0.25,-0.5,-0.75,-1.0,-1.25,-1.5,-1.75,-2.0,-2.25,-2.5,-2.75,-3.,-3.25])
agelist = np.array([8.95,9.,9.05,9.1,9.15,9.2,9.25,9.3,9.35,9.4,9.45,9.5,9.55,9.6,9.65,9.7,9.75,9.8,9.85,9.9,9.95,10.,10.05,10.1])
nmet  = len(metlist)
nage  = len(agelist)

# for MIST
nfilt = 7
filter_names = np.array([r"$F062$",r"$F087$",r"$F106$",r"$F129$",r"$F146$",r"$F158$",r"$F184$"])
# filter indexes to be used below
filter_index_dict = {"F062":9,"F087":10,"F106":11,"F129":12,"F146":13,"F158":14,"F184":15}
# 5-sigma point source detection limit for 10^3 sec exposure in each band
ps_detect_5slim = np.array([27.8, 27.5, 27.4, 27.3, 27.6, 27.3, 26.8])
path_to_isos = "../../mist_isos"

We first will define a function that will tell us the brightest object in a given Equivalent Evolutionary Phase (EEP). This is a term used by the MIST Isochrone team to determine a certain phase of stellar evolution, so here we are only using the MIST Isochrones. Unlike our other notebooks, in order to use this function you will have needed to download the MIST Isochrones and put them in `iso_path` which will be given to the function below so that it can load the isochrones. You should make sure that the naming of your isochrone files follows the convention of the files in the function written below and that you have downloaded the isochrones for the same range of ages and metallicities that we use here.

In [4]:
# This is a function to determine the brightest absolute magnitude
# in a given population feature, determined by the ranges in EEP 
# that are provided, and a given filter, determined by the 
# filter_index variable. This function determines this magnitude for
# each metallicity and age in the mlist and alist provided.
# iso_path tells the function where your isochrones are
def get_absM(mlist,alist,filter_indices,eep_low,eep_high,iso_path):
    outM = np.zeros((mlist.shape[0],alist.shape[0],len(filter_indices)))
    for (i,met) in enumerate(mlist):
        if met>-0.1:
            (eep,age,imass,T,L) = np.loadtxt("%s/mist_fehp%1.2f.iso.cmd"%(iso_path,abs(met)),usecols=[0,1,2,4,6]).T
            M = np.loadtxt("%s/mist_fehp%1.2f.iso.cmd"%(iso_path,abs(met)),usecols=filter_indices).T
        else:
            (eep,age,imass,T,L) = np.loadtxt("%s/mist_fehm%1.2f.iso.cmd"%(iso_path,abs(met)),usecols=[0,1,2,4,6]).T
            M = np.loadtxt("%s/mist_fehm%1.2f.iso.cmd"%(iso_path,abs(met)),usecols=filter_indices).T

        for (j,a) in enumerate(alist):

            sids = np.intersect1d(np.where(age>a-0.01),np.where(age<a+0.01))
            feature_ids = np.intersect1d(sids,np.where(np.logical_and(eep>=eep_low,eep<=eep_high)))
            
            for k in range(len(filter_indices)):
                # need to make sure that this feature is actually 
                # apparent at this age and metallicity
                if len(feature_ids)>0:
                    outM[i][j][k] = min(M[k][feature_ids])
                else:
                    outM[i][j][k] = -1000
    return(outM)

We can apply this function to get, for example, the absolute magnitude of the TRGB over a range of metallicities and ages. In our work, we define the TRGB as $560 \leq {\rm EEP}\leq 580$. Note the use of the pre-defined (above) index dictionary for a given filter. This dictionary may change based on what filters you selected when downloading the MIST isochrones, keep this in mind. Your `path_to_isos` variable should also not necessarily be the same as mine, set above.

Here we will get the absolute magnitude of the TRGB in the $F158$ band.

In [5]:
find = [9,10,11,12,13,14,15]
msto_mags = get_absM(metlist,agelist,find,1,495,path_to_isos)
trgb_mags = get_absM(metlist,agelist,find,560,580,path_to_isos)
hb_mags = get_absM(metlist,agelist,find,630,640,path_to_isos)

In [6]:
np.save("../data/msto_mags",msto_mags)
np.save("../data/hb_mags",hb_mags)
np.save("../data/trgb_mags",trgb_mags)